### Purpose:
- Use a directory or directories of mel spectrograms to generate ABGQI predictions from ABGQI-CNN. 
- Each mel spec directory will result in one csv. 
- Each row of the csv corresponds to a single mel spec.

### Notes
- uses a directory of the 6 demo mels (combined in a single dir)
- adjust CPUs to desired amount

In [1]:
import tensorflow as tf
'''
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1
'''
import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import os
import pathlib
import glob
import joblib
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
AUTOTUNE = tf.data.experimental.AUTOTUNE
tf.__version__

'2.3.0'

# 0. USER INPUTS

In [82]:
#output directory
out_dir = '../results/'

# trained models
wd ='../../cnn_training/results/'
checkpoint_path = os.path.join(wd, 'ABGQI-CNN') # IMGNET + S2L CVFOLDS

# data dir for images to predict class
data_path = '../../melspec_generation/results/all_melspecs/'

# number of cores
cpus = 2

print(checkpoint_path)
print(out_dir)

../../cnn_training/results/ABGQI-CNN
../results/


In [5]:
# dimensions of our images    
img_width, img_height, img_depth = 224, 224, 3 # 224 pixels x 224 pixels x 3 bands (RGB)

In [6]:
# number of images to predict at once 
batch_size = 50

## Functions

In [7]:
# function to read in a file path
def process_path(file_path, IMG_HEIGHT, IMG_WIDTH):
    # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img, IMG_HEIGHT, IMG_WIDTH)
    return img #, label

# function that interpretes image after being provided a path in process_path
def decode_img(img, IMG_HEIGHT, IMG_WIDTH):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  return tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])

# 1. Load trained model checkpoint

In [10]:
# LOAD MODEL
model = tf.keras.models.load_model(checkpoint_path)

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.40_224 (Functi (None, 7, 7, 1792)        4363712   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1792)              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 8965      
Total params: 4,372,677
Trainable params: 4,324,741
Non-trainable params: 47,936
_________________________________________________________________


# 2. Set up sigmoid prediction

In [61]:
png_dirs = []
for filename in os.listdir(data_path):
    if os.path.isdir(os.path.join(data_path, filename)):
        png_dirs.append(filename)
if len(png_dirs) == 0: 
    png_dirs.append(data_path)

In [62]:
png_dirs

['../../melspec_generation/results/all_melspecs/']

In [51]:
out_dir

'../results/demo_inference'

In [52]:
print(checkpoint_path)

../../cnn_training/results/ABGQI-CNN


In [90]:
# iterate through each directory (class or wav dir with mfccs within)
for i,f in enumerate(png_dirs):
    temp_dir = f
    pngs = glob.glob(temp_dir + "*.png")   
    temp_class = temp_dir.split('/')[-2] # png parent dir
    print()
    print(out_dir)
    print("Number of pngs", len(pngs)) 
    
    # check if the parent output dir exists
    if(os.path.exists(out_dir)):
        print("directory already exists. Moving to predictions...")            
    else:
        os.mkdir(out_dir)
        print("Created :", out_dir)
        
    # list to hold each dir of predictions (either all pngs in class or mfccs in wav)
    sigmoid_pred_lst = []
    
    # file path for class pkl
    out_path_temp = os.path.join(os.path.join(out_dir, "sigmoid",  temp_class))
    print(out_path_temp)
    
    # iterate through each melspec in file directory
    for j in range(len(pngs)):      
        temp_png = pngs[j] # png        
        
        #check if pkl (prediction) for file exists
        if os.path.isfile(out_path_temp):
            pass # skip file 
        
        # enter prediction 
        else:
            # process image
            img_ = process_path(temp_png, IMG_HEIGHT = img_height, IMG_WIDTH = img_width)
            img_ = tf.reshape(img_, shape= (1, img_height, img_width, img_depth))
            
            # get predictions
            pred = model.predict(img_, verbose=0, steps=1, callbacks=None, max_queue_size=10, 
                  workers=cpus, use_multiprocessing=False)

            sigmoid_pred = tf.math.sigmoid(pred).numpy()
            sigmoid_pred_lst.append(sigmoid_pred)
      
    # save label preds
    flat_sigmoid = [item for sublist in sigmoid_pred_lst for item in sublist]
    df_sigmoid = pd.DataFrame(flat_sigmoid)
    df_sigmoid.columns = ["Anthrophony", "Biophony", "Geophony", "Other", "Interference"]
    
    # write directory csv 
    df_sigmoid.to_csv(os.path.join(out_dir, temp_class + '.csv'), index = False)     
    print('Saved this pkl at:',os.path.join(out_dir, temp_class), "\nLength was:",len(sigmoid_pred_lst))


../results/
Number of pngs 6
directory already exists. Moving to predictions...
../results/sigmoid\all_melspecs
Saved this pkl at: ../results/all_melspecs 
Length was: 6
